In [ ]:
"""IMPORT EVERYTHING.. """
# Data manipulation
import pandas as pd
import numpy as np

# Quantitative analysis / indicators
import talib
import backtrader as bt

# # Machine learning / statistics
# import scipy
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression

# # Visualization
# import matplotlib.pyplot as plt
# import seaborn as sns
import mplfinance as mpf

# # Jupyter Notebook magic for inline plots
# %matplotlib inline


In [ ]:
df = pd.read_parquet("../Src/filtered.XAUUSD-M1.parquet", engine="pyarrow")

# print(df.head(3))

In [ ]:
from tafm import EnhancedTechnicalAnalyzer, create_analyzer, IndicatorConfig

df.rename(columns={
    'OPEN': 'Open',
    'HIGH': 'High',
    'LOW': 'Low',
    'CLOSE': 'Close',
    'VOL': 'Volume'
}, inplace=True)

def example_basic_indicators(df):
    """Example: Adding technical indicators"""

    # Initialize analyzer
    analyzer = create_analyzer(df)

    # Add indicators using fluent interface
    analyzer.add_indicator(
        IndicatorConfig(name='EMA', period=21)
    ).add_indicator(
        IndicatorConfig(name='EMA', period=50)
    ).add_indicator(
        IndicatorConfig(name='RSI', period=14)
    ).add_indicator(
        IndicatorConfig(name='MACD', fast_period=12, slow_period=26, signal_period=9)
    )

    analyzer.crossed_down('EMA_21', 'EMA_50', 'Cross_e2150')

    # View results
    print("DataFrame with indicators:")
    print(analyzer.df[['Close', 'EMA_21', 'EMA_50', 'RSI_14', 'MACD']].tail())

    # Get summary
    print("\nOperations Summary:")
    print(analyzer.summary())

    return analyzer

signal = example_basic_indicators(df)

In [ ]:
from resampler import OHLCVResampler

# Resample to 1 hour - all columns preserved
resampler = OHLCVResampler(default_agg='last')

result = resampler.resample(signal.df, '1min', '2024-11-01', '2024-11-02')

result.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
}, inplace=True)

print(f"\nResampled shape: {result.shape}")

# result

In [ ]:
from stock_indicators import Quote, indicators

# Convert DataFrame to a list of Quote objects
quotes = [
    Quote(
        date,
        row.open,
        row.high,
        row.low,
        row.close,
        row.volume
    )
    for date, row in result.iterrows()
]

In [ ]:
fractal_results = indicators.get_fractal(quotes, 15)

result['fractal_bear']= [r.fractal_bear for r in fractal_results]
result['fractal_bull']= [r.fractal_bull for r in fractal_results]

In [ ]:
result

In [ ]:

from ChartterX5 import Chartter

chartter = Chartter(config={
    'chart_type': 'candle',
    'style': 'charles',
    'volume': False
})

chartter._setup_tradingview_theme()

cnf = {
        'title': 'NONE',
        'style': chartter.tradingview_style,
        'volume': False,
        'show_nontrading': False,
        'datetime_format': '%Y-%m-%d',
        'xlabel': '',
        'ylabel': '',
        'xrotation': 0
    }
# result.head(3)

In [ ]:
fractal_bear_plot = result['fractal_bear'].copy()
fractal_bear_plot[fractal_bear_plot.isna()] = np.nan

fractal_bull_plot = result['fractal_bull'].copy()
fractal_bull_plot[fractal_bull_plot.isna()] = np.nan


Cross_e2150 = result[result['Cross_e2150']==1].index

vline_series = result['Cross_e2150'].replace(0, np.nan)

In [ ]:
# Create base chart
fig, axes =chartter.plot(
    result,
    addplot=[
        mpf.make_addplot(result['RSI_14'], panel=1, color='purple', ylabel='RSI'),
        mpf.make_addplot(result['EMA_21'], panel=0, color='white', ylabel='RSI'),
        mpf.make_addplot(result['EMA_50'], panel=0, color='blue', ylabel='RSI'),
        mpf.make_addplot(fractal_bear_plot, panel=0, type='scatter', markersize=21, marker='.', color='red'),
        mpf.make_addplot(fractal_bull_plot, panel=0, type='scatter', markersize=21, marker='.', color='Yellow'),
        mpf.make_addplot(vline_series,panel=1,type='scatter',markersize=20,color='orange')
    ],
    config=cnf,
    vlines = {
        'vlines': Cross_e2150.to_list(),
        'colors': 'orange',
        'linewidths': 0.9,
        'linestyle': '--',
        'alpha': 0.7
    },
    returnfig=True
)

In [ ]:
result['last_fractal'] = result['fractal_bear']

# Forward-fill to carry the last fractal value forward
result['last_fractal_ffill'] = result['last_fractal'].ffill()

result['entry'] = np.nan
result['sl'] = np.nan


# When Cross_e2150 == 1, assign the close as entry
result.loc[result['Cross_e2150'] == 1, 'entry'] = result['close']
result.loc[result['Cross_e2150'] == 1, 'sl'] = result['last_fractal_ffill']

result['risk'] = result['sl'].astype('float') - result['entry']

result['target'] = result['entry'] - result['risk'] * 1


result.loc[result['Cross_e2150'] == 1].head().T

In [ ]:
import mplfinance as mpf
import matplotlib.patches as patches

# Ensure numeric (float) types
result[['entry', 'sl', 'target']] = result[['entry', 'sl', 'target']].astype(float)

# Keep only valid rows
entries = result.dropna(subset=['entry', 'sl', 'target'])

ax = axes[0]

# Draw rectangles for each entry
for dt, row in entries.iterrows():
    x = result.index.get_loc(dt)
    width = 5
    height = row['target'] - row['sl']
    rect = patches.Rectangle(
        (x - width/2, row['sl']),
        width,
        height,
        linewidth=1,
        edgecolor='green',
        facecolor='green',
        alpha=0.3
    )
    ax.add_patch(rect)

fig
